In [1]:
import csv
import random
from utils import *
from collections import defaultdict
import time
import matplotlib.pyplot as plt
#import seaborn as sns
import numpy as np

In [2]:
adjacency_list, edge_list = create_graph_from_csv("email-Eu-core.csv")

In [ ]:
'''
TODO

See if functions are doable

'''

In [ ]:

no_edges = len(edge_list)  #No. of edges

prob_del = 0.2      #Probability to delete edge

current_graph = {}
current_edge_list = []

available_edge_list = np.random.permutation(edge_list)

stream_length = 2*no_edges

for i in range(stream_length):
    
    #Insertion
    if available_edge_list: #
        current_edge_list.append(available_edge_list[i])
        u = available_edge_list[i][0]
        v = available_edge_list[i][1]
        if u not in current_graph.keys():
            current_graph[u] = OptList()
        current_graph[u].insert(v)
        if v not in current_graph.keys():
            current_graph[v] = OptList()
        current_graph[v].insert(u)
        available_edge_list.pop(0)
        
        '''
        Code for SDD and PIVOT goes here
        '''
        
    else:
        #We have run out of edges to insert
        edge_to_delete = np.random.choice(current_edge_list)
        
        u = edge_to_delete[0]
        v = edge_to_delete[1]
        current_graph[u].remove(v)
        current_graph[v].remove(u)
        
        available_edge_list.extend(edge_to_delete)
        current_edge_list.remove(edge_to_delete)
    
        
    
    
    if np.random.binomial(1,prob_del):
        #Deletion
        edge_to_delete = np.random.choice(current_edge_list)
        
        u = edge_to_delete[0]
        v = edge_to_delete[1]
        current_graph[u].remove(v)
        current_graph[v].remove(u)
        
        available_edge_list.extend(edge_to_delete)
        current_edge_list.remove(edge_to_delete)
        



